<a href="https://colab.research.google.com/github/ShinAsakawa/xerion/blob/master/notebooks/2019wbaihandson_tiny_char_rnn_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [第2回失語症ハンズオン](https://wba-initiative.org/3411/)

<div>
    <center><img src="https://wba-initiative.org/wp-content/uploads/2015/05/logo.png" style="width:29%"></center>
</div>
    

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 15/Mar/2019<br>
 MIT license
</div>

---

# tiny 文字RNN による文字予測

[Original](https://github.com/ekzhang/char-rnn-keras) <https://github.com/ekzhang/char-rnn-keras>

- <font size="+2" color='blue'>文字ベースの RNN を使って簡単な予測を行ってみます</font>

In [0]:
# 必要なライブラリの輸入 `import`
import json
import os
from pathlib import Path
import sys
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [0]:
# モデルの作成。n_cells と n_layers とを変更して性能を確認してください
def build_model(batch_size, seq_len, vocab_size, n_cells=128, n_layers=2):
    model = Sequential()
    model.add(Embedding(vocab_size, n_cells, batch_input_shape=(batch_size, seq_len)))
    for i in range(n_layers):
        model.add(LSTM(n_cells, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(vocab_size)))
    model.add(Activation('softmax'))
    return model

In [0]:
# ここでは簡単のため，ごく短い文章を学習させてみます。日本国憲法第9条
text = "".join('第９条 日本国民は、正義と秩序を基調とする国際平和を誠実に希求し、\n')
text += "".join('国権の発動たる戦争と、武力による威嚇又は武力の行使は、国際紛争を解決する手段としては、永久にこれを放棄する。\n')
text += "".join('２項 前項の目的を達するため、陸海空軍その他の戦力は、これを保持しない。国の交戦権は、これを認めない。\n')

chr2idx = {ch: i for (i, ch) in enumerate(sorted(list(set(text))))}
idx2chr = {idx:char for char, idx in chr2idx.items()}
vocab_size = len(chr2idx)

Data = np.asarray([chr2idx[c] for c in text], dtype=np.int32)
for d in Data:
    print(idx2chr[d], end='')

In [0]:
BATCH_SIZE = 14  # ミニバッチのサイズ
SEQ_LENGTH = 8   # 一つの系列の長さ

def make_batches(D, vocab_size, batch_size=BATCH_SIZE, seq_length=SEQ_LENGTH):
    #length = X.shape[0]
    n_chars = D.shape[0] // batch_size

    for start in range(0, n_chars - seq_length, seq_length):
        X = np.zeros((batch_size, seq_length))
        Y = np.zeros((batch_size, seq_length, vocab_size))
        for idx in range(0, batch_size):
            for i in range(0, seq_length):
                X[idx, i] = D[n_chars * idx + start + i]
                Y[idx, i, D[n_chars * idx + start + i + 1]] = 1
        yield X, Y
        
for (X, Y) in make_batches(Data, vocab_size, batch_size=BATCH_SIZE, seq_length=SEQ_LENGTH):
    print(X.shape, Y.shape)

In [0]:
# モデルを作成してコンパイルします
model = build_model(BATCH_SIZE, SEQ_LENGTH, vocab_size)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])    

In [0]:
epochs = 20    # エポック数だけ学習します。20 回では精度が十分ではありません。何度か繰り返します
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    losses, accs = [], []
    for i, (X, Y) in enumerate(make_batches(Data, vocab_size, batch_size=BATCH_SIZE, seq_length=SEQ_LENGTH)):
    #for i, (X, Y) in enumerate(make_batches(Data, vocab_size)):
        loss, acc = model.train_on_batch(X, Y)
        print('Batch {}: loss = {:.4f}, acc = {:.5f}'.format(i + 1, loss, acc))
        losses.append(loss)
        accs.append(acc)
        
    yhat = model.predict(X)
    for y in yhat:
        for c in y:
            print(idx2chr[np.argmax(c)], end='')
        print('/', end='')
    print()

In [0]:
# 実際に予測できたか試してみましょう
a = model.predict(X)
for x in a:
    for l in x:
        print(idx2chr[np.argmax(l)], end='')
    print('/',end='')
print()

In [0]:
# 結果を保存します。保存する必要もないですが，一応
model.save_weights('j_const9_weights.h5')
model.save('j_const9_model.h5')